In [1]:
from pyspark import SparkContext
import pyspark
import time

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

21/12/03 11:16:46 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.1.143.47 instead (on interface en0)
21/12/03 11:16:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/03 11:16:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/03 11:16:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/03 11:16:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/03 11:16:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
business_sf = sc.textFile(
    "../data/SF_business/filtered_registered_business_sf.tsv")
supervisor_sf = sc.textFile("../data/SF_business/supervisor_sf.tsv")

In [3]:
partition_ct = 4

## 1. Create an RDD which has partition_ct as the number of partitions  in a format of (zip, supervisor_id) order by zip.

In [7]:
supervisor_sf.map(lambda x: x.split('\t'))\
            .sortByKey(numPartitions=partition_ct)

[('94102', '8'),
 ('94102', '6'),
 ('94102', '3'),
 ('94102', '5'),
 ('94103', '8'),
 ('94103', '9'),
 ('94103', '10'),
 ('94103', '6'),
 ('94103', '3'),
 ('94103', '5'),
 ('94104', '6'),
 ('94104', '3'),
 ('94105', '6'),
 ('94105', '3'),
 ('94107', '10'),
 ('94107', '6'),
 ('94108', '6'),
 ('94108', '3'),
 ('94109', '2'),
 ('94109', '6'),
 ('94109', '3'),
 ('94109', '5'),
 ('94110', '8'),
 ('94110', '11'),
 ('94110', '9'),
 ('94110', '10'),
 ('94111', '6'),
 ('94111', '3'),
 ('94112', '7'),
 ('94112', '8'),
 ('94112', '11'),
 ('94112', '9'),
 ('94112', '10'),
 ('94114', '7'),
 ('94114', '8'),
 ('94114', '5'),
 ('94115', '2'),
 ('94115', '1'),
 ('94115', '5'),
 ('94116', '7'),
 ('94116', '4'),
 ('94117', '1'),
 ('94117', '7'),
 ('94117', '8'),
 ('94117', '5'),
 ('94118', '2'),
 ('94118', '1'),
 ('94118', '5'),
 ('94121', '2'),
 ('94121', '1'),
 ('94122', '1'),
 ('94122', '7'),
 ('94122', '5'),
 ('94122', '4'),
 ('94123', '2'),
 ('94124', '9'),
 ('94124', '10'),
 ('94127', '7'),
 ('9412

In [4]:
sorted_zip_supervisor_id = supervisor_sf.map(lambda x : x.split('\t'))\
                                        .map(lambda x : (int(x[0]), int(x[1])))\
                                        .sortByKey(numPartitions=partition_ct)

## how the data are organized?
- Do same keys locate in the same partition?

In [5]:
sorted_zip_supervisor_id.glom().collect()

[[(94102, 8),
  (94102, 6),
  (94102, 3),
  (94102, 5),
  (94103, 8),
  (94103, 9),
  (94103, 10),
  (94103, 6),
  (94103, 3),
  (94103, 5),
  (94104, 6),
  (94104, 3),
  (94105, 6),
  (94105, 3),
  (94107, 10),
  (94107, 6),
  (94108, 6),
  (94108, 3),
  (94109, 2),
  (94109, 6),
  (94109, 3),
  (94109, 5)],
 [(94110, 8),
  (94110, 11),
  (94110, 9),
  (94110, 10),
  (94111, 6),
  (94111, 3),
  (94112, 7),
  (94112, 8),
  (94112, 11),
  (94112, 9),
  (94112, 10),
  (94114, 7),
  (94114, 8),
  (94114, 5),
  (94115, 2),
  (94115, 1),
  (94115, 5)],
 [(94116, 7),
  (94116, 4),
  (94117, 1),
  (94117, 7),
  (94117, 8),
  (94117, 5),
  (94118, 2),
  (94118, 1),
  (94118, 5),
  (94121, 2),
  (94121, 1),
  (94122, 1),
  (94122, 7),
  (94122, 5),
  (94122, 4),
  (94123, 2),
  (94124, 9),
  (94124, 10)],
 [(94127, 7),
  (94127, 8),
  (94127, 11),
  (94129, 2),
  (94130, 6),
  (94131, 7),
  (94131, 8),
  (94131, 5),
  (94132, 7),
  (94132, 11),
  (94132, 4),
  (94133, 2),
  (94133, 3),
  (94134

## 2. Create an RDD which has partition_ct as the number of partitions in a format of (zip, [name, street, city, zip]) partitioned by zip (hash).

In [12]:
business_sf.map(lambda x: x.split('\t'))\
            .filter(lambda x: len(x[0])!=0)\
            .map(lambda x: (int(x[0]),x[1:]))\
            .partitionBy(numPartitions=partition_ct).take(5)

[(94124,
  ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
 (94108, ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
 (94108, ['Stephens Institute Inc', '740 Taylor St', 'San Francisco', 'CA']),
 (94108,
  ['"Fugazi Travel Agency, Inc."',
   '170 Grant Ave 4th Fl',
   'San Francisco',
   'CA']),
 (94132, ['Agid Hyman D', '452 Gellert Dr', 'San Francisco', 'CA'])]

In [13]:
partitioned_zip_business = business_sf.map(lambda x : x.split('\t'))\
                                      .filter(lambda x: len(x[0]) > 0)\
                                      .map(lambda x : (int(x[0]), x[1:]))\
                                      .partitionBy(numPartitions=partition_ct)

## how the data are organized?
- Do same keys locate in the same partition?

In [14]:
partitioned_zip_business.glom().collect()

[[(94124,
   ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
  (94108, ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
  (94108, ['Stephens Institute Inc', '740 Taylor St', 'San Francisco', 'CA']),
  (94108,
   ['"Fugazi Travel Agency, Inc."',
    '170 Grant Ave 4th Fl',
    'San Francisco',
    'CA']),
  (94132, ['Agid Hyman D', '452 Gellert Dr', 'San Francisco', 'CA']),
  (94108, ['Felton Donald & Nancy', '445 Stockton St', 'San Francisco', 'CA']),
  (94108, ['Felton Donald & Nancy', '445 Stockton St', 'San Francisco', 'CA']),
  (94124, ['Anresco Inc', '1370 Van Dyke Ave', 'San Francisco', 'CA']),
  (94124, ['Anresco Inc', '1370 Van Dyke Ave', 'San Francisco', 'CA']),
  (94108,
   ['Anthonys Shoe Service Inc', '340 Kearny St', 'San Francisco', 'CA']),
  (94112, ['Antonchuk Richard A', '1117 Ocean Ave', 'San Francisco', 'CA']),
  (94112, ['Antonchuk Richard A', '1125 Ocean Ave', 'San Francisco', 'CA']),
  (94124, ['Arnold & Egan Mfg Co', '1

## 3. Join sorted_zip_supervisor_id and partitioned_zip_business efficiently for any sorted_zip_supervisor_id. (assume that the joined RDDs will be repeatedly used)

In [8]:
sorted_zip_supervisor_id.cache()
partitioned_zip_business.cache()

In [9]:
start = time.time()
joined_supervisor_business = sorted_zip_supervisor_id.leftOuterJoin(partitioned_zip_business)
joined_supervisor_business.collect()
print(time.time() - start)

2.840353012084961


# Custom Partitioner

## 1) For the range of supervisor ids, create RDD of (supervisor_id, [business name, street, city, state]) within the particion_ct based on the supervisor_id serially.
- Ex. if partition_ct is 3, and supervisor ids are [1, 2, 3, 4, 5, 6, 7, 8, 9] => first partition should have pairs with supervisor ids of [1,2,3], second paritition should have pairs with supervisor ids of [4,5,6], third partition should have pairs with supervisor ids of [7,8,9]

In [10]:
supervisor_business = joined_supervisor_business.values()

In [11]:
supervisor_ids = joined_supervisor_business\
    .values()\
    .keys()\
    .distinct()\
    .sortBy(lambda x: x)\
    .collect()

In [12]:
supervisor_ids

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [13]:
def custom_range_partition(key):
    min_id = supervisor_ids[0]
    max_id = supervisor_ids[-1]
    partition_size = (max_id - min_id + 1)/partition_ct
    return int((key - min_id)/partition_size)

In [14]:
supervisor_business_custom_range = supervisor_business.partitionBy(numPartitions=partition_ct,
                                                                   partitionFunc=custom_range_partition)
supervisor_business_custom_range.saveAsTextFile('supervisor_business_custom_range')

## 2)  Store pairs of (supervisor_id, [business name, street, city, state]) within 2 partitions, based on whether the supervisor_id is even or odd.

In [15]:
def odd_even_partitioner(key):
    return key%2

In [16]:
supervisor_business_odd_even = supervisor_business.partitionBy(2, odd_even_partitioner)
supervisor_business_odd_even.saveAsTextFile('supervisor_business_odd_even')

In [16]:
sc.stop()